# CIFAR-10 Dataset

The CIFAR-10 dataset consists of __60000 32x32__ colour images in __10 classes__, with 6000 images per class. There are 50000 training images and 10000 test images.

Here are the classes in the dataset, as well as 10 random images from each: 

<img src="cifar-10-random.png">

# Prep Data

In [ ]:
import os
import urllib.request
import hashlib
import tarfile
import pickle
import numpy as np
import gzip

Download approximately 170MB of data.

In [ ]:
url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
fname = 'cifar-10-python.tar.gz'

if not os.path.isfile(fname):
    urllib.request.urlretrieve(url, fname)

fsum = hashlib.md5(open(fname, 'rb').read()).hexdigest()
assert fsum == 'c58f30108f718f92721af3b95e74349a'

Load dataset

In [ ]:
def extract(tfile, member):
    buff = tfile.extractfile(member)
    pdata = pickle.load(buff, encoding='latin1')
    features = pdata['data']
    features = features.reshape([len(pdata['data']), 3, 32, 32])
    features = features.transpose(0, 2, 3, 1)
    labels = pdata['labels']
    return features, labels

with tarfile.open(fname, 'r:gz') as tfile:
    members = tfile.getmembers()
    x1, y1 = extract(tfile, 'cifar-10-batches-py/data_batch_1')
    x2, y2 = extract(tfile, 'cifar-10-batches-py/data_batch_2')
    x3, y3 = extract(tfile, 'cifar-10-batches-py/data_batch_3')
    x4, y4 = extract(tfile, 'cifar-10-batches-py/data_batch_4')
    x5, y5 = extract(tfile, 'cifar-10-batches-py/data_batch_5')    
    
    train_x_raw = np.concatenate([x1, x2, x3, x4, x5])
    train_y_raw = np.concatenate([y1, y2, y3, y4, y5])
    
    test_x_raw, test_y_raw = extract(tfile, 'cifar-10-batches-py/test_batch')
    
    del x1, x2, x3, x4, x5, y1, y2, y3, y4, y5

### Preprocess

Scale

In [ ]:
train_x.std()

In [ ]:
train_x.mean()

In [ ]:
train_x_mean = train_x.mean()

In [ ]:
train_x_std = train_x.std()

In [ ]:
train_x = (train_x - train_x_mean) / train_x_std
test_x = (test_x - train_x_mean) / train_x_std

In [ ]:
train_x = train_x_raw / 255
test_x = test_x_raw / 255

Dummies

In [ ]:
def onehot(data_0123):
    N = len(data_0123)
    res = np.zeros([N, 10])
    res[range(N), data_0123] = 1
    return res

train_y = onehot(train_y_raw)
test_y = onehot(test_y_raw)

Split - dataset is already shuffled

In [ ]:
idx = int(len(train_x)*0.1)

valid_x = train_x[-idx:]
valid_y = train_y[-idx:]

train_x = train_x[:-idx]
train_y = train_y[:-idx]

### Save to drive

In [ ]:
#raise  # Sentinel
with gzip.open('CIFAR-10.gz', 'wb') as gfile:
    pickle_dict = {'train_x': train_x, 'train_y': train_y,
                   'valid_x': valid_x, 'valid_y': valid_y,
                   'test_x': test_x, 'test_y': test_y }
    pickle.dump(pickle_dict, gfile)

# Checkpoint

In [ ]:
import gzip
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tables

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
with tf.Session(config=config) as sess:
    devs = sess.list_devices()
    print('\n'.join([x.name for x in devs]))

In [ ]:
with gzip.open('CIFAR-10.gz', 'rb') as gfile:
    pickle_dict = pickle.load(gfile)
    train_x = pickle_dict['train_x']
    train_y = pickle_dict['train_y']
    valid_x = pickle_dict['valid_x']
    valid_y = pickle_dict['valid_y']
    test_x = pickle_dict['test_x']
    test_y = pickle_dict['test_y']
    del pickle_dict  # free up memory

# Explore Data

In [ ]:
lab2txt = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

idx = 43
plt.imshow(train_x[idx])
print(lab2txt[np.argmax(train_y[idx])])

# Model

### High-Level Model

In [ ]:
tf.reset_default_graph()

xi = tf.placeholder(name='xi', shape=[None, 32, 32, 3], dtype=tf.float32)
yi = tf.placeholder(name='yi', shape=[None, 10], dtype=tf.float32)
kp = tf.placeholder(name='kp', shape=[], dtype=tf.float32)

model = tf.layers.conv2d(xi,
    filters=32, kernel_size=[5, 5], strides=[1, 1], padding='SAME', activation=tf.nn.relu, name='Conv_1')
model = tf.layers.max_pooling2d(model,
    pool_size=[2, 2], strides=[2, 2], padding='SAME', name='Pool_1')

model = tf.layers.conv2d(model,
    filters=64, kernel_size=[5, 5], strides=[1, 1], padding='SAME', activation=tf.nn.relu, name='Conv_2')
model = tf.layers.max_pooling2d(model,
    pool_size=[2, 2], strides=[2, 2], padding='SAME', name='Pool_2')

model = tf.layers.conv2d(model,
    filters=128, kernel_size=[5, 5], strides=[1, 1], padding='SAME', activation=tf.nn.relu, name='Conv_3')
model = tf.layers.max_pooling2d(model,
    pool_size=[2, 2], strides=[2, 2], padding='SAME', name='Pool_3')

model = tf.layers.flatten(model)
model = tf.layers.dense(model, 128, activation=tf.nn.relu, name='Dense_1')
model = tf.nn.dropout(model, kp)
logits = tf.layers.dense(model, 10, activation=None, name='Logits')

probabilities = tf.nn.softmax(logits)

loss = tf.losses.softmax_cross_entropy(yi, logits)

optimizer = tf.train.AdamOptimizer()
grads_and_vars = optimizer.compute_gradients(loss)
train_op = optimizer.apply_gradients(grads_and_vars)

is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(yi, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

graph = tf.get_default_graph()

In [ ]:
dict_inout = {
    'batch_x' : xi[0:batch_save,:,:,:],
    'batch_y' : yi[0:batch_save,:],
    'probabilities' : probabilities[0:batch_save,:]
}

In [ ]:
dict_conv_1 = {
'W': graph.get_tensor_by_name('Conv_1/kernel:0'),
'b': graph.get_tensor_by_name('Conv_1/bias:0'),
'dW': graph.get_tensor_by_name('gradients/Conv_1/Conv2D_grad/tuple/control_dependency_1:0'),
'db': graph.get_tensor_by_name('gradients/Conv_1/BiasAdd_grad/tuple/control_dependency_1:0'),
'z': graph.get_tensor_by_name('Conv_1/BiasAdd:0')[0:batch_save,:,:,:],
}

In [ ]:
dict_conv_2 = {
'W': graph.get_tensor_by_name('Conv_2/kernel:0'),
'b': graph.get_tensor_by_name('Conv_2/bias:0'),
'dW': graph.get_tensor_by_name('gradients/Conv_2/Conv2D_grad/tuple/control_dependency_1:0'),
'db': graph.get_tensor_by_name('gradients/Conv_2/BiasAdd_grad/tuple/control_dependency_1:0'),
'z': graph.get_tensor_by_name('Conv_2/BiasAdd:0')[0:batch_save,:,:,:],
}

In [ ]:
dict_conv_3 = {
'W': graph.get_tensor_by_name('Conv_3/kernel:0'),
'b': graph.get_tensor_by_name('Conv_3/bias:0'),
'dW': graph.get_tensor_by_name('gradients/Conv_3/Conv2D_grad/tuple/control_dependency_1:0'),
'db': graph.get_tensor_by_name('gradients/Conv_3/BiasAdd_grad/tuple/control_dependency_1:0'),
'z': graph.get_tensor_by_name('Conv_3/BiasAdd:0')[0:batch_save,:,:,:],
}

In [ ]:
dict_dense_1 = {
'W': graph.get_tensor_by_name('Dense_1/kernel:0'),
'b': graph.get_tensor_by_name('Dense_1/bias:0'),
'dW': graph.get_tensor_by_name('gradients/Dense_1/MatMul_grad/tuple/control_dependency_1:0'),
'db': graph.get_tensor_by_name('gradients/Dense_1/BiasAdd_grad/tuple/control_dependency_1:0'),
'z': graph.get_tensor_by_name('Dense_1/BiasAdd:0')[0:batch_save,:],
}

In [ ]:
dict_logits = {
'W': graph.get_tensor_by_name('Logits/kernel:0'),
'b': graph.get_tensor_by_name('Logits/bias:0'),
'dW': graph.get_tensor_by_name('gradients/Logits/MatMul_grad/tuple/control_dependency_1:0'),
'db': graph.get_tensor_by_name('gradients/Logits/BiasAdd_grad/tuple/control_dependency_1:0'),
'z': graph.get_tensor_by_name('Logits/BiasAdd:0')[0:batch_save,:],
}

dict_metrics = {
    'loss': loss,
    'accuracy': accuracy,
}

In [ ]:
dict_layers = {
    'inout': dict_inout,
    'conv_1': dict_conv_1,
    'conv_2': dict_conv_2,
    'conv_3': dict_conv_3,
    'dense_1': dict_dense_1,
    'logits': dict_logits,
    'metrics': dict_metrics,
}

In [ ]:
tf.trainable_variables()

In [ ]:
grads_and_vars

In [ ]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    out2 = sess.run( tf.get_default_graph().get_tensor_by_name('Conv_1/kernel:0') )
    out1 = sess.run( tf.trainable_variables()[0] )
    

In [ ]:
np.alltrue(out1 == out2)

In [ ]:
def create_log(filename, all_layers_dict):
    with tables.open_file(filename, mode='w') as f:
        for layer_name, layer_dict in all_layers_dict.items():
            group = f.create_group('/', layer_name)
            for tensor_name, tensor in layer_dict.items():
                tensor_shape = tensor.get_shape().as_list()
                if len(tensor_shape) > 0 and tensor_shape[0] is None:
                    tensor_shape[0] = batch_save
                f.create_earray(group, tensor_name, atom=tables.Float32Atom(), shape=[0, *tensor_shape])
                # print(layer_name, tensor_name, tensor_shape)

In [ ]:
def print_log(filename):
    with tables.open_file(filename, mode='a') as f:
        for group in f.root:
            print(group._v_name)
            for tensor_earray in group:
                print('{0:10} {1:20} {2:30}'.format(group._v_name, tensor_earray._v_name, str(tensor_earray.shape)),
                      '{0:12.2f}'.format(tensor_earray[:].nbytes/1e6), 'MB')

In [ ]:
def append_log(filename, all_layers_dict):
    with tables.open_file(filename, mode='a') as f:
        for layer_name, layer_dict in all_layers_dict.items():
            group = f.root[layer_name]
            for tensor_name, tensor_data in layer_dict.items():
                group[tensor_name].append( np.expand_dims(tensor_data, axis=0) )

In [ ]:
filename = 'outarray.h5'

In [ ]:
create_log(filename, dict_layers)

In [ ]:
print_log(filename)

In [ ]:
with tables.open_file('outarray.h5', mode='r') as f:
    print(f)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logdir='tf_log', graph=sess.graph)
    writer.flush()

### Train Loop

Hyperparameters

In [ ]:
epochs = 5 # 40
batch_size = 250
batch_save = 5
keep_probability = 0.75

Train Loop

In [ ]:


saver = tf.train.Saver()
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        for batch_i, idx in enumerate(range(0, len(train_x), batch_size)):
            
            xb = train_x[idx:idx+batch_size]
            yb = train_y[idx:idx+batch_size]
            
            summary, _, _ = sess.run([dict_layers, train_op, loss], feed_dict={xi: xb, yi: yb, kp: keep_probability})
            
            append_log(filename, summary)
            
        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
        train_cost, train_acc = sess.run([loss, accuracy], feed_dict={xi: xb, yi: yb, kp: 1.0})
        valid_cost, valid_acc = sess.run([loss, accuracy], feed_dict={xi: valid_x, yi: valid_y, kp: 1.0})
        print('tc, vc, tacc, vacc', train_cost, valid_cost, train_acc, valid_acc)
        
    saver.save(sess, './model.ckpt')

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './model.ckpt')
    
    train_cost, train_acc = sess.run([loss, accuracy], feed_dict={xi: train_x[:10000], yi: train_y[:10000], kp: 1.0})
    valid_cost, valid_acc = sess.run([loss, accuracy], feed_dict={xi: valid_x, yi: valid_y, kp: 1.0})
    test_cost, test_acc   = sess.run([loss, accuracy], feed_dict={xi: test_x,  yi: test_y,  kp: 1.0})
    print('Train Set Accuracy:', train_acc)
    print('Validation Set Acc:', valid_acc)
    print('Test Set Accuracy: ', test_acc)

# Image Recognition - Code

<span style="color:red">
    TODO: 
</span>

In [ ]:
import tensorflow as tf

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
with tf.Session(config=config) as sess:
    devs = sess.list_devices()
    print('\n'.join([x.name for x in devs]))

In [ ]:
import PIL
import PIL.ImageOps
import requests
import io
import os
import matplotlib.pyplot as plt

In [ ]:
def get_image_path():
    PATH = '~/'
    full_path = os.path.expanduser(PATH)
    all_in_home = os.listdir(full_path)

    screenshot_paths = []
    for file_name in all_in_home:
        file_path = os.path.join(full_path, file_name)
        if os.path.isfile(file_path):
            if file_name.startswith('Screenshot'):
                screenshot_paths.append(file_path)

    image_path = sorted(screenshot_paths)[-1]
    return image_path

In [ ]:
def get_input_data(image_path):
    # fetch image
    img_orig = PIL.Image.open(image_path).convert('RGB')
    w, h = img_orig.size
    ms = min(img_orig.size)
    img_cropped = img_orig.crop([w//2-ms//2, h//2-ms//2, w//2+ms//2, h//2+ms//2])

    # thumbnail 32x32
    thumb = PIL.ImageOps.fit(img_cropped, [32, 32], PIL.Image.ANTIALIAS)

    # numpy array 32x32x3
    arr3 = np.array(thumb, dtype=np.float32) / 255

    # array 1x32x32x3
    arr4 = np.reshape(arr3, [-1, *arr.shape])
    return arr3, arr4, img_orig

In [ ]:
def tf_get_probabilities(x_data):    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, './model.ckpt')

        out_logits, out_prob = sess.run([logits, probabilities], feed_dict={xi: x_data, kp:1.0})

#         print('logits:', out_logits)
#         print('probabilities', out_prob)
#         print('argmax', np.argmax(out_prob))
#         print('category:', lab2txt[np.argmax(out_prob)])
        
        return out_prob.flatten()

In [ ]:
def display_image_predictions(features, labels, predictions):
    n_classes = 10
    label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    label_binarizer = LabelBinarizer()
    label_binarizer.fit(range(n_classes))
    label_ids = label_binarizer.inverse_transform(np.array(labels))

    fig, axies = plt.subplots(nrows=4, ncols=2)
    fig.tight_layout()
    fig.suptitle('Softmax Predictions', fontsize=20, y=1.1)

    n_predictions = 3
    margin = 0.05
    ind = np.arange(n_predictions)
    width = (1. - 2. * margin) / n_predictions

    for image_i, (feature, label_id, pred_indicies, pred_values) in enumerate(zip(features, label_ids, predictions.indices, predictions.values)):
        pred_names = [label_names[pred_i] for pred_i in pred_indicies]
        correct_name = label_names[label_id]

        axies[image_i][0].imshow(feature)
        axies[image_i][0].set_title(correct_name)
        axies[image_i][0].set_axis_off()

        axies[image_i][1].barh(ind + margin, pred_values[::-1], width)
        axies[image_i][1].set_yticks(ind + margin)
        axies[image_i][1].set_yticklabels(pred_names[::-1])
        axies[image_i][1].set_xticks([0, 0.5, 1.0])

In [ ]:
colors = ['blue'] * 10
colors[3] = 'red'
colors

In [ ]:
lab2txt = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def run_neural_net():
    x_3d, x_4d, img = get_input_data(get_image_path())
    prob = tf_get_probabilities(x_4d)

    fig = plt.figure(figsize=[16,6])
    axi = fig.add_subplot(131)
    axi.imshow(img)
    axi.set_title('Original Image')
    
    axn = fig.add_subplot(132)
    axn.imshow(x_4d[0])
    axn.set_title('Neural Network Sees')

    axp = fig.add_subplot(133)
    colors = ['blue'] * 10
    colors[np.argmax(prob)] = 'green'
    axp.barh(range(10), prob,   align='center', color=colors)
    axp.set_yticks(range(10))
    strings = map('{:.2f}'.format, prob*100)
    labels = map('  '.join, zip(lab2txt, strings))
    axp.set_yticklabels(labels)
    axp.set_title('Predictions')

    plt.tight_layout()
    plt.show()

# run_neural_net()

In [ ]:
lab2txt = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

idx = 120
x = np.expand_dims(train_x[idx], axis=0)

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './model.ckpt')
    
    out_logits = sess.run([logits], feed_dict={xi: x, kp:1.0})
    
    index = np.argmax(out_logits)
    print()
    print(lab2txt[index])
    plt.imshow(train_x[idx])
    
    
#     train_cost, train_acc = sess.run([loss, accuracy], feed_dict={xi: train_x[:10000], yi: train_y[:10000], kp: 1.0})
#     valid_cost, valid_acc = sess.run([loss, accuracy], feed_dict={xi: valid_x, yi: valid_y, kp: 1.0})
#     test_cost, test_acc   = sess.run([loss, accuracy], feed_dict={xi: test_x,  yi: test_y,  kp: 1.0})
#     print('Train Set Accuracy:', train_acc)
#     print('Validation Set Acc:', valid_acc)
#     print('Test Set Accuracy: ', test_acc)

# Image Recognition

Neural Net can recognize following categories:

<img src="cifar-10-random.png">

To run this:

1. Take screenshot with SHIFT-PrintScreen
2. Run cell bellow

In [ ]:
run_neural_net()